# Part 1: Fetching, Cleaning, & Storing Stock Data in Azure Cloud Data Warehouse

**Step 1: Install and Load Packages**

In [9]:
%pip install pandas numpy requests

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 64 kB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 209 kB 5.4 MB/s eta 0:00:01
     |████████████████████████████████| 131 kB 21.9 MB/s eta 0:00:01
     |████████████████████████████████| 71 kB 20.0 MB/s eta 0:00:01
     |████████████████████████████████| 159 kB 15.6 MB/s eta 0:00:01
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [10]:
# Import libraries
import pandas as pd
import numpy as np
import sys
import requests
from datetime import datetime


/Users/kpaudel/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


**Step 2: Load & Clean Stock Data from API**

In [12]:
# Load API key from config file
sys.path.append('.')
from config import ALPHA_VANTAGE_API_KEY


In [13]:


def fetch_stock_data(symbol, api_key, start_year=2010):
    """
    Fetch stock data from Alpha Vantage API from 2010 to current.
    Returns DataFrame with open, mid_day (average of high/low), and close prices.
    """
    # Alpha Vantage API endpoint for daily adjusted time series
    url = "https://www.alphavantage.co/query"
    params = {
        "function": "TIME_SERIES_DAILY_ADJUSTED",
        "symbol": symbol,
        "apikey": api_key,
        "outputsize": "full",  # Get full historical data
        "datatype": "json"
    }
    
    print(f"Fetching data for {symbol}...")
    response = requests.get(url, params=params)
    data = response.json()
    
    # Check for API errors
    if "Error Message" in data:
        print(f"Error: {data['Error Message']}")
        return None
    if "Note" in data:
        print(f"Note: {data['Note']}")
        return None
    
    # Extract time series data
    time_series = data.get("Time Series (Daily)", {})
    
    # Convert to DataFrame
    df = pd.DataFrame.from_dict(time_series, orient='index')
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    
    # Rename columns
    df.columns = [col.split('. ')[1] if '. ' in col else col for col in df.columns]
    
    # Select and rename columns to get open, high, low, close
    df = df[['open', 'high', 'low', 'close']].astype(float)
    
    # Calculate mid-day price as average of high and low
    df['mid_day'] = (df['high'] + df['low']) / 2
    
    # Filter from start_year to current
    df = df[df.index >= f"{start_year}-01-01"]
    
    # Select only open, mid_day, and close as requested
    df = df[['open', 'mid_day', 'close']]
    
    print(f"Data loaded: {len(df)} days from {df.index.min().date()} to {df.index.max().date()}")
    return df



In [ ]:
# Example: Fetch data for Apple (AAPL)
stock_data = fetch_stock_data("AAPL", ALPHA_VANTAGE_API_KEY, start_year=2010)

# Display the data
if stock_data is not None:
    print("\nFirst 5 rows:")
    print(stock_data.head())
    print("\nLast 5 rows:")
    print(stock_data.tail())
    print(f"\nData shape: {stock_data.shape}")
    print(f"\nData types:")
    print(stock_data.dtypes)
